In [1]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
%run $header_path

C:\Users\raphael.wargon\Documents\GitHub\sorting_univ\script/headers/


In [2]:
scanR_path 

'E:\\scanR'

In [3]:
works_au_af = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'works_au_af.parquet')
               .filter( func.col('publication_year').between(1950,2020) )
              )

In [4]:
df_authors = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'authors.parquet')
              )

In [5]:
df_authors

DataFrame[affiliations: array<struct<institution:struct<country_code:string,country_id:string,display_name:string,id:string,lineage:array<string>,ror:string,type:string,type_id:string>,years:array<bigint>>>, cited_by_count: bigint, counts_by_year: array<struct<cited_by_count:bigint,oa_works_count:bigint,works_count:bigint,year:bigint>>, created_date: string, display_name: string, display_name_alternatives: array<string>, id: string, ids: struct<openalex:string,orcid:string,scopus:string>, last_known_institutions: array<struct<country_code:string,country_id:string,display_name:string,id:string,lineage:array<string>,ror:string,type:string,type_id:string>>, orcid: string, summary_stats: struct<2yr_cited_by_count:bigint,2yr_h_index:bigint,2yr_i10_index:bigint,2yr_mean_citedness:double,2yr_works_count:bigint,cited_by_count:bigint,h_index:bigint,i10_index:bigint,works_count:bigint>, topic_share: array<struct<display_name:string,domain:struct<display_name:string,id:string>,field:struct<displa

In [ ]:
#df_au_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + '\\authors.parquet')

# Build set of French authors

In [ ]:
set_french_authors = (works_au_af
                      .filter(func.col('publication_year').between(1997,2020))
                      .filter(func.col('country')=='FR')
                      .groupBy('author_id')
                      .agg(func.max('publication_year').alias('last_year'),
                           func.min('publication_year').alias('entry_year'),
                           func.countDistinct('work_id').alias('n_pub_in_FR'),
                           func.countDistinct('publication_year').alias('n_y_in_FR')
                          )
                      .filter( ~(func.col('last_year')==func.col('entry_year'))
                               & (func.col('n_y_in_FR')>2)
                               & (func.col('n_pub_in_FR')>2)
                             )
                     )
#set_french_authors.cache()
#set_french_authors.count()

In [ ]:
(set_french_authors.filter(func.col('author_id')=="A5084984675")
                      .join(works_au_af, on = ['author_id'], how = 'left')).show()

In [9]:
set_french_authors.show()

+-----------+---------+----------+-----------+---------+
|  author_id|last_year|entry_year|n_pub_in_FR|n_y_in_FR|
+-----------+---------+----------+-----------+---------+
|A5081892619|     2010|      2002|         11|        5|
|A5079653947|     2020|      2006|         56|       13|
|A5105641933|     2020|      1998|        183|       23|
|A5007411109|     2020|      1997|       1138|       24|
|A5027058032|     2020|      2003|        146|       16|
|A5053414030|     2006|      1997|         10|        7|
|A5028937897|     2020|      1998|        273|       22|
|A5008496995|     2020|      2003|         40|       13|
|A5028912390|     2020|      2001|         66|       18|
|A5061749308|     2020|      1997|        133|       23|
|A5087508730|     2020|      2002|         26|       13|
|A5019424031|     2020|      1999|         76|       17|
|A5039654491|     2017|      2011|          3|        3|
|A5089185306|     2020|      1999|        112|       21|
|A5019988390|     2020|      19

In [19]:
set_french_authors.write.mode('overwrite').parquet('file:\\' + save_path + 'set_french_authors.parquet')

# Load set of French authors

In [5]:
set_french_authors = spark.read.parquet('file:\\' + save_path + 'set_french_authors.parquet')
set_french_authors.count()

380346

In [7]:
works_au_af_subset = (set_french_authors
                      .join(works_au_af.select('author_id', 'inst_id', 'publication_year', 'language', 'source_id','topics', 'citations'), on = ['author_id'], how = 'left')
                     )
works_au_af_subset.cache()

works_au_af_subset.show()

# Build author-level characteristics


In [8]:
au_level_char = (works_au_af_subset
              .groupBy('author_id','publication_year').agg(func.max((func.col('country')=='FR').cast('int')).alias('in_FR'),
                                                          func.max((func.col('country')!='FR').cast('int')).alias('abroad'))
              .groupBy('author_id').agg(func.min('publication_year').alias('entry_year'),
                                        func.max('publication_year').alias('last_year'),
                                        func.sum('in_FR').alias('all_y_in_FR'), func.sum('abroad').alias('years_abroad'))
                )
#au_level_char.cache()
#au_level_char.show()

In [36]:
au_level_char.filter(func.col('author_id')=="A5084984675").show()

+-----------+----------+---------+-----------+------------+
|  author_id|entry_year|last_year|all_y_in_FR|years_abroad|
+-----------+----------+---------+-----------+------------+
|A5084984675|      1985|     2020|         28|          33|
+-----------+----------+---------+-----------+------------+



In [9]:
au_fields = (set_french_authors
             .join(df_authors
                   .select(func.regexp_replace('id', 'https://openalex.org/','').alias('author_id'), func.explode(func.col('topic_share')).alias('topic_share'),
                          func.col('display_name').alias('author_name'))
                   , on = ['author_id'], how = 'inner')
             .withColumn('max_topic_share', func.max(func.col('topic_share.value')).over(Window.partitionBy('author_id')))
             .filter(func.col('max_topic_share') <= func.col('topic_share.value'))
             .withColumn('field', func.substring(func.lower(func.col('topic_share.field.display_name')), 1, 4))
             .withColumn('main_field', func.concat_ws(',', func.array_sort(func.collect_set(func.col('field')).over(Window.partitionBy('author_id')))))
             .select('author_id', 'main_field','author_name')
             #.groupBy('author_id','main_field','author_name')
             #.pivot('field')
             #.agg(func.lit(1))
             #.fillna(0)
            )
au_fields

DataFrame[author_id: string, main_field: string, author_name: string]

In [9]:
#au_level_char.count()

In [10]:
au_level_char = (au_level_char
                 .filter((func.col('entry_year')<2020) & (func.col('entry_year')>1954))
                 .join(au_fields, on = ['author_id'], how = 'left')
                 .distinct()
                )

In [11]:
#au_level_char.show()

In [12]:
au_level_char.write.mode('overwrite').parquet('file:\\' + save_path + 'au_level_char.parquet')

In [ ]:
#spark.catalog.clearCache()

# Load author-level characteristics

In [19]:
au_level_char= spark.read.parquet('file:\\' + save_path + 'au_level_char.parquet')

In [13]:
#au_level_char = spark.read.parquet('file:\\' + save_path + 'au_level_char.parquet')

In [16]:
#au_level_char.filter(func.col('author_id')=="A5087461705").show()

In [17]:
#(au_level_char.groupBy((func.floor(func.col('entry_year')/5)*5)
#                       .alias('entry_year'))
#.agg(*[func.mean(col).alias(col) for col in ['last_year',"all_y_in_FR", "years_abroad"]]).sort('entry_year')).show(100)

In [18]:
#(au_level_char
# .groupBy(((func.floor(func.col('all_y_in_FR')/(func.col('last_year')-func.col('entry_year')+1)*50)*2).alias('share_y_in_fr')))
# .count()
# .sort('share_y_in_fr')).show(100)

# Build institution-level information

In [19]:
#spark.catalog.clearCache()

In [20]:
#works_au_af_subset = (set_french_authors
#                      .join(works_au_af.select('work_id','author_id', 'inst_id', 'publication_year', 'language', 'source_id'), on = ['author_id'], how = 'left')
#                     )
#works_au_af_subset.cache()#

#works_au_af_subset.show()

In [ ]:
inst= spark.read.parquet('file:\\' + save_path + 'inst_fr.parquet')

In [13]:
works_au_af_subset = (works_au_af_subset
                 # .withColumn('inst_id', func.expr(expr_inst_id))
                 # .join(inst.drop('country').distinct(), on = ['inst_id'])
                  .withColumn('inst_id', func.expr(expr_inst_id))
                  .withColumn('inst_id', func.when(func.col('inst_id')== "I4210140029", "I197518295").otherwise(func.col('inst_id')))
                  .join(inst.distinct(), on = ['inst_id'])
                     )
works_au_af_subset.cache()
works_au_af_subset.show()

+-----------+-----------+---------+----------+-----------+---------+-----------+----------------+--------+-----------+--------------------+----------+-------+--------------------+-----+-------+----+
|    inst_id|  author_id|last_year|entry_year|n_pub_in_FR|n_y_in_FR|    work_id|publication_year|language|  source_id|           inst_name| inst_type|country|              parent|fused|uni_pub|cnrs|
+-----------+-----------+---------+----------+-----------+---------+-----------+----------------+--------+-----------+--------------------+----------+-------+--------------------+-----+-------+----+
|I4210149539|A5000157973|     2020|      2003|         80|       16|W3003814814|            2019|      fr|S4306402512|Laboratoire d'Ant...|  facility|     FR|[{"parent_id":"I1...|    0|      0|   1|
|I4210149539|A5000157973|     2020|      2003|         80|       16|W4378736409|            2016|      en|S4306402512|Laboratoire d'Ant...|  facility|     FR|[{"parent_id":"I1...|    0|      0|   1|
|I129

In [23]:
works_au_af_subset.distinct(), on = ['work_id'], how = 'left').write.mode('overwrite').parquet('file:\\' + save_path + 'works_au_af_subset.parquet') 

In [7]:
works_au_af_subset = spark.read.parquet('file:\\' + save_path + 'works_au_af_subset.parquet') 

In [8]:
works_au_af_subset.show()

+-----------+-----------+-----------+---------+----------+-----------+---------+----------------+--------+-----------+--------------------+----------+-------+--------------------+-----+-------+----+---------+
|    work_id|    inst_id|  author_id|last_year|entry_year|n_pub_in_FR|n_y_in_FR|publication_year|language|  source_id|           inst_name| inst_type|country|              parent|fused|uni_pub|cnrs|citations|
+-----------+-----------+-----------+---------+----------+-----------+---------+----------------+--------+-----------+--------------------+----------+-------+--------------------+-----+-------+----+---------+
| W100149476| I237446400|A5031545966|     2017|      1998|         23|       10|            2011|      fr|       NULL|Institut Superieu...| education|     FR|                [{}]|    0|      0|   0|        0|
| W100149476| I157674565|A5031545966|     2017|      1998|         23|       10|            2011|      fr|       NULL| University of Liège| education|     BE|[{"par

In [9]:
au_inst_period = (works_au_af_subset
                    .groupBy('author_id','inst_id',func.col('publication_year').alias('year'))
                    .agg(func.countDistinct('work_id').alias('n_works'),
                        func.first('parent').alias('parent'), func.first('country').alias('country'),
                        func.first('entry_year').alias('entry_year'))
                    .withColumn('n_appearances', func.collect_set(func.struct(func.col('year').alias('year'),
                                                                            func.col('n_works').alias('n_works'))).over(Window.partitionBy('author_id', 'inst_id')))
                    
                    .withColumn('n_appearances', func.array_compact(func.transform(func.col('n_appearances'), 
                                                      lambda x: func.when( (x.year >= func.col('year')-5) & (x.year <= func.col('year')+5), x.n_works.cast('int')))))
                    .withColumn('n_appearances', func.aggregate(func.col('n_appearances'), func.lit(0), lambda acc, x: acc + x))
                    .withColumn('n_appearances_ctry', func.sum('n_works').over(Window.partitionBy('author_id','country')))
                    
                    .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
                    .withColumn('parent', func.regexp_replace(func.col('parent'), '\[',''))
                    .withColumn('parent', func.regexp_replace(func.col('parent'), '\]',''))
                    .withColumn('parent', func.when(func.col('parent').contains(';'), func.split(func.col('parent'), ';'))
                                .otherwise(func.array(func.col('parent'))))
                    .withColumn('parent', func.transform(func.col('parent'), lambda x: func.from_json(x, func.schema_of_json('{"parent_id":"","parent_name":"","parent_type":""}'))))
                    .withColumn('parent', func.array_compact(func.transform(func.col('parent'), lambda x : func.when(x.parent_id != func.col('inst_id'), x))))
                    .withColumn('all_parent_ids', func.array_compact(func.transform(func.array_distinct(func.flatten(func.collect_set(func.col('parent.parent_id')).over(Window.partitionBy('year')))), lambda x: func.regexp_replace(x, 'https://openalex.org/', ''))))
                    .withColumn('test_drop', (func.array_contains(func.col('all_parent_ids'), func.col('inst_id'))).cast('int'))
                    .filter( (func.col('test_drop')==0) | (func.col('test_drop').isNull()))  
                    .withColumn('inst_id', func.when(  ((func.col('n_appearances')>1) & (func.col('n_appearances_ctry') >1) )
                                                     | (func.col('year') == func.col('entry_year'))
                                                       , func.col('inst_id')))
                    .withColumn('inst_id', func.when(func.col('inst_id').isNull(), func.last(func.col('inst_id'), ignorenulls = True).over(Window.partitionBy('author_id','year'))).otherwise(func.col('inst_id')))
                    .select('author_id','inst_id','year')
                    )

<>:14: SyntaxWarning: invalid escape sequence '\}'
<>:14: SyntaxWarning: invalid escape sequence '\}'
<>:15: SyntaxWarning: invalid escape sequence '\['
<>:16: SyntaxWarning: invalid escape sequence '\]'
<>:14: SyntaxWarning: invalid escape sequence '\}'
<>:14: SyntaxWarning: invalid escape sequence '\}'
<>:15: SyntaxWarning: invalid escape sequence '\['
<>:16: SyntaxWarning: invalid escape sequence '\]'
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\2487976817.py:14: SyntaxWarning: invalid escape sequence '\}'
  .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\2487976817.py:14: SyntaxWarning: invalid escape sequence '\}'
  .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\2487976817.py:15: SyntaxWarning: invalid escape sequence '\['
  .withColumn('parent', func.regexp_replace(func.col('parent'), 

In [10]:
au_inst_period.cache()
au_inst_period.show()

+-----------+-----------+----+
|  author_id|    inst_id|year|
+-----------+-----------+----+
|A5000010047|I2801259928|1959|
|A5000010047| I166337079|1959|
|A5000027759|       NULL|1959|
|A5000090806|  I78577930|1959|
|A5000409387|I4210107423|1959|
|A5000570208|I4210163092|1959|
|A5000601191|       NULL|1959|
|A5000726633|       NULL|1959|
|A5000776451|       NULL|1959|
|A5000971455|       NULL|1959|
|A5001283845| I111979921|1959|
|A5001307276|       NULL|1959|
|A5001307276|       NULL|1959|
|A5001307276|       NULL|1959|
|A5001307276|       NULL|1959|
|A5001590383|  I87208433|1959|
|A5001910430|       NULL|1959|
|A5001969191|       NULL|1959|
|A5002136143|       NULL|1959|
|A5002146860|  I53964585|1959|
+-----------+-----------+----+
only showing top 20 rows



In [11]:
au_inst_period.count()

5853066

In [12]:
au_inst_period = (au_inst_period
                 .withColumn('new_inst_period', func.when((func.lag('year').over(Window.partitionBy('author_id','inst_id').orderBy('year')).isNull())
                                                            | (func.lag('year').over(Window.partitionBy('author_id','inst_id').orderBy('year')) == func.col('year')-1
                                                               ), 0)
                                                      .otherwise(1))
                  .withColumn('period_inst',1+ func.sum(func.col('new_inst_period')).over(Window.partitionBy("author_id", "inst_id").orderBy('year')))    
                 )
#au_inst_period.cache()
#au_inst_period.show()

In [13]:
au_inst_period = (au_inst_period
                  .select(*au_inst_period.columns,
                          (func.max(func.col('year')).over(Window.partitionBy('period_inst','inst_id','author_id'))).alias('last_y_inst_period'),
                          (func.min(func.col('year')).over(Window.partitionBy('period_inst','inst_id','author_id'))).alias('first_y_inst_period'),
                          func.size(func.collect_set('inst_id').over(Window.partitionBy('year','author_id'))).alias('n_inst_y')
                         )
                 )
#au_inst_period.cache()
#au_inst_period.show()

In [14]:
journals_ranking = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'journals_ranking_year.parquet')
                    .withColumn('source_id', func.regexp_replace(func.col('source_id'), 'https://openalex.org/', ''))
                   )
journals_ranking.show()

+-----------+----+-----------+------------------+------------------+
|  source_id|year|source_type|   rank_source_pct|   abs_rank_source|
+-----------+----+-----------+------------------+------------------+
| S100028386|2000|    journal|0.8187134502923976|             217.0|
| S100045985|1957|    journal|0.8255395683453237|              98.0|
| S100045985|1985|    journal|0.9227373068432672|             176.0|
| S100045985|2001|    journal|0.9590356722853783|             210.0|
| S100062655|2013|    journal|0.8979432336514213|1085.4476190476191|
|S1000839972|1999|    journal|0.8367626886145405|             238.0|
|S1000839972|2017|    journal|0.8949784017278618|             388.0|
|S1000944648|2016|    journal| 0.807972611464417|2328.6081081081084|
|S1000944648|2017|    journal|0.8922782643712877|            2132.0|
|S1000944648|2019|    journal|0.6022198843945348|          6325.625|
| S100105777|2016|    journal|0.5854789417204764| 5389.076923076924|
|  S10012645|1987|    journal|0.88

In [17]:
au_pub_y = (works_au_af_subset
            .withColumnRenamed('publication_year','year')
            .join(journals_ranking, on = ['source_id','year'], how = 'left')
            .select('author_id','work_id', 'year','citations','rank_source_pct')
            .distinct()
            .groupBy('author_id','year')
            .agg(func.countDistinct('work_id').alias('publications_raw'),
                 func.sum(func.col('citations')).alias('citations_raw'),

                 func.mean('rank_source_pct').alias('avg_rank_source_raw'),
                 func.sum((func.col('rank_source_pct')<= 0.5).cast('int')).alias('nr_source_btm_50pct_raw'),
                 func.sum((func.col('rank_source_pct').between(0.5,0.9)).cast('int')).alias('nr_source_mid_40pct_raw'),
                 func.sum((func.col('rank_source_pct')>= 0.8).cast('int')).alias('nr_source_top_20pct_raw'),
                 func.sum((func.col('rank_source_pct')>= 0.9).cast('int')).alias('nr_source_top_10pct_raw'),
                 func.sum((func.col('rank_source_pct')>= 0.95).cast('int')).alias('nr_source_top_5pct_raw'),

                
                )
           )

In [20]:
au_inst_period = (au_inst_period
                  .join(au_pub_y, on = ['author_id','year'], how = 'left')
                  .withColumn('publications', func.col('publications_raw')/func.col('n_inst_y'))
                  .withColumn('citations', func.col('citations_raw')/func.col('n_inst_y'))
                  .join(inst.distinct(), on = ['inst_id'])
                  .join(au_level_char, on = ['author_id'])
                  .withColumn('entry_cohort', func.when(func.col('entry_year')>=2000, func.col('entry_year'))
                              .otherwise(func.round(func.col('entry_year')/5)*5))
                  #.withColumn('n_inst_id_sample', func.size(func.collect_set('inst_id').over(Window.partitionBy('author_id'))))
                  #.withColumn('n_au_several_inst', func.size(func.collect_set(func.when(func.col('n_inst_id_sample')>1, func.col('author_id')))
                  #                                           .over(Window.partitionBy('inst_id','main_field'))))
                  #.withColumn('n_au_by_inst_field', func.size(func.collect_set(func.col('author_id'))
                  #                                           .over(Window.partitionBy('inst_id','main_field'))))
                  #.withColumn('n_by_field', func.size(func.collect_set('author_id').over(Window.partitionBy('main_field'))))
                  #.withColumn('n_y_in_sample', func.size(func.collect_set('year').over(Window.partitionBy('author_id'))))
                  ##.filter(func.col('year').between(2000,2020))
                 )
au_inst_period.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- inst_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- new_inst_period: integer (nullable = false)
 |-- period_inst: long (nullable = true)
 |-- last_y_inst_period: integer (nullable = true)
 |-- first_y_inst_period: integer (nullable = true)
 |-- n_inst_y: integer (nullable = false)
 |-- publications_raw: long (nullable = true)
 |-- citations_raw: long (nullable = true)
 |-- avg_rank_source_raw: double (nullable = true)
 |-- nr_source_btm_50pct_raw: long (nullable = true)
 |-- nr_source_mid_40pct_raw: long (nullable = true)
 |-- nr_source_top_20pct_raw: long (nullable = true)
 |-- nr_source_top_10pct_raw: long (nullable = true)
 |-- nr_source_top_5pct_raw: long (nullable = true)
 |-- publications: double (nullable = true)
 |-- citations: double (nullable = true)
 |-- inst_name: string (nullable = true)
 |-- inst_type: string (nullable = true)
 |-- country: string (nullable = true)
 |-- parent: string (nullable = t

In [20]:
#au_inst_period.show()

In [21]:
au_inst_period.write.mode('overwrite').parquet('file:\\' + save_path + 'panel_non_sphericised.parquet')

In [27]:
spark.catalog.clearCache()

In [22]:
au_inst_period = spark.read.format('parquet').load('file:\\' + save_path + 'panel_non_sphericised.parquet')

In [11]:
au_inst_period = au_inst_period_smoothed

In [23]:
au_inst_period.select('author_id').distinct().count()

333840

In [25]:
au_inst_period.filter(func.col('author_id') =='A5036205978').sort('year').show()

+-----------+-----------+----+---------------+-----------+------------------+-------------------+--------+----------------+-------------+--------------------+---------+-------+--------------------+-----+-------+----+----------+---------+-----------+------------+----------+-----------------+------------+------------+--------------------+------------+---------+
|  author_id|    inst_id|year|new_inst_period|period_inst|last_y_inst_period|first_y_inst_period|n_inst_y|publications_raw|citations_raw|           inst_name|inst_type|country|              parent|fused|uni_pub|cnrs|entry_year|last_year|all_y_in_FR|years_abroad|main_field|      author_name|entry_cohort|period_total|  inst_set_this_year|publications|citations|
+-----------+-----------+----+---------------+-----------+------------------+-------------------+--------+----------------+-------------+--------------------+---------+-------+--------------------+-----+-------+----+----------+---------+-----------+------------+----------+---

In [24]:
au_all_years = (au_inst_period
                .groupBy('author_id')
                .agg(func.min('year').alias('entry_year'), func.max('year').alias('last_year'))
                .select('author_id',
                        func.explode(func.sequence(func.col('entry_year'), func.col('last_year'))).alias('year')
                       )
               )
au_all_years.filter(func.col('author_id') =='A5036205978').show()


+-----------+----+
|  author_id|year|
+-----------+----+
|A5036205978|2006|
|A5036205978|2007|
|A5036205978|2008|
|A5036205978|2009|
|A5036205978|2010|
|A5036205978|2011|
|A5036205978|2012|
|A5036205978|2013|
|A5036205978|2014|
|A5036205978|2015|
|A5036205978|2016|
|A5036205978|2017|
|A5036205978|2018|
|A5036205978|2019|
|A5036205978|2020|
+-----------+----+



In [25]:
au_period = (au_inst_period
             .groupBy('author_id', 'year')
             .agg(func.array_sort(  func.collect_set('inst_id')).alias('inst_set_this_year'))
                      .withColumn('lag_inst_set_this_year', func.lag('inst_set_this_year').over(Window.partitionBy('author_id').orderBy('year')))
                      .withColumn('new_period', (
                          ( (func.col('lag_inst_set_this_year')!=func.col('inst_set_this_year')) | (func.col('lag_inst_set_this_year').isNull()) )
                            ).cast('int') )
                      .withColumn('period_total', func.sum('new_period').over(Window.partitionBy('author_id').orderBy('year')))
                     )

In [45]:
(au_period.filter(func.col('author_id') =='A5036205978')
 .select('year','inst_set_this_year','lag_inst_set_this_year', 'new_period', 'period_total')
 .sort('year','period_total')).collect()

[Row(year=2006, inst_set_this_year=['I4210149295'], lag_inst_set_this_year=None, new_period=1, period_total=1),
 Row(year=2010, inst_set_this_year=['I4210149295'], lag_inst_set_this_year=['I4210149295'], new_period=0, period_total=1),
 Row(year=2011, inst_set_this_year=['I4210123768', 'I4210149295'], lag_inst_set_this_year=['I4210149295'], new_period=1, period_total=2),
 Row(year=2013, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768', 'I4210149295'], new_period=1, period_total=3),
 Row(year=2017, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768'], new_period=0, period_total=3),
 Row(year=2019, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768'], new_period=0, period_total=3),
 Row(year=2020, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768'], new_period=0, period_total=3)]

In [46]:
au_period.filter(func.col('author_id')=="A5036205978").sort('year').collect()

[Row(author_id='A5036205978', year=2006, inst_set_this_year=['I4210149295'], lag_inst_set_this_year=None, new_period=1, period_total=1),
 Row(author_id='A5036205978', year=2010, inst_set_this_year=['I4210149295'], lag_inst_set_this_year=['I4210149295'], new_period=0, period_total=1),
 Row(author_id='A5036205978', year=2011, inst_set_this_year=['I4210123768', 'I4210149295'], lag_inst_set_this_year=['I4210149295'], new_period=1, period_total=2),
 Row(author_id='A5036205978', year=2013, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768', 'I4210149295'], new_period=1, period_total=3),
 Row(author_id='A5036205978', year=2017, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768'], new_period=0, period_total=3),
 Row(author_id='A5036205978', year=2019, inst_set_this_year=['I4210123768'], lag_inst_set_this_year=['I4210123768'], new_period=0, period_total=3),
 Row(author_id='A5036205978', year=2020, inst_set_this_year=['I4210123768'], lag_inst_set_thi

In [26]:
au_period = (au_period
             .withColumn('duration_period', func.sequence(func.min('year').over(Window.partitionBy('author_id', 'period_total')),
                                                          func.max('year').over(Window.partitionBy('author_id', 'period_total'))
                                                        )
                        )
             .withColumn('year', func.explode(func.col('duration_period')))
             .select('author_id', 'year', 'inst_set_this_year','period_total')
             .distinct()
            )
au_period.filter(func.col('author_id')=="A5036205978").sort('year').collect()

[Row(author_id='A5036205978', year=2006, inst_set_this_year=['I4210149295'], period_total=1),
 Row(author_id='A5036205978', year=2007, inst_set_this_year=['I4210149295'], period_total=1),
 Row(author_id='A5036205978', year=2008, inst_set_this_year=['I4210149295'], period_total=1),
 Row(author_id='A5036205978', year=2009, inst_set_this_year=['I4210149295'], period_total=1),
 Row(author_id='A5036205978', year=2010, inst_set_this_year=['I4210149295'], period_total=1),
 Row(author_id='A5036205978', year=2011, inst_set_this_year=['I4210123768', 'I4210149295'], period_total=2),
 Row(author_id='A5036205978', year=2013, inst_set_this_year=['I4210123768'], period_total=3),
 Row(author_id='A5036205978', year=2014, inst_set_this_year=['I4210123768'], period_total=3),
 Row(author_id='A5036205978', year=2015, inst_set_this_year=['I4210123768'], period_total=3),
 Row(author_id='A5036205978', year=2016, inst_set_this_year=['I4210123768'], period_total=3),
 Row(author_id='A5036205978', year=2017, inst

In [27]:
au_period = (au_all_years
             .join(au_period, how = 'left', on = ['author_id','year'])
             .withColumn('inst_set_this_year', 
                         func.when(func.col('inst_set_this_year').isNull(),
                                   func.last(func.col('inst_set_this_year'), ignorenulls = True)
                                   .over(Window.partitionBy('author_id').orderBy('year')))
                         .otherwise(func.col('inst_set_this_year'))
                        )
             .withColumn('period_total', 
                         func.when(func.col('period_total').isNull(),
                                   func.last(func.col('period_total'), ignorenulls = True)
                                   .over(Window.partitionBy('author_id').orderBy('year')))
                         .otherwise(func.col('period_total'))
                        )
             .withColumn('n_inst', func.size(func.col('inst_set_this_year')))
            )
au_period.filter(func.col('author_id')=="A5036205978").sort('year').collect()

[Row(author_id='A5036205978', year=2006, inst_set_this_year=['I4210149295'], period_total=1, n_inst=1),
 Row(author_id='A5036205978', year=2007, inst_set_this_year=['I4210149295'], period_total=1, n_inst=1),
 Row(author_id='A5036205978', year=2008, inst_set_this_year=['I4210149295'], period_total=1, n_inst=1),
 Row(author_id='A5036205978', year=2009, inst_set_this_year=['I4210149295'], period_total=1, n_inst=1),
 Row(author_id='A5036205978', year=2010, inst_set_this_year=['I4210149295'], period_total=1, n_inst=1),
 Row(author_id='A5036205978', year=2011, inst_set_this_year=['I4210123768', 'I4210149295'], period_total=2, n_inst=2),
 Row(author_id='A5036205978', year=2012, inst_set_this_year=['I4210123768', 'I4210149295'], period_total=2, n_inst=2),
 Row(author_id='A5036205978', year=2013, inst_set_this_year=['I4210123768'], period_total=3, n_inst=1),
 Row(author_id='A5036205978', year=2014, inst_set_this_year=['I4210123768'], period_total=3, n_inst=1),
 Row(author_id='A5036205978', year

In [28]:
au_inst_period_smoothed = (au_inst_period.drop('period_total','inst_set_this_year')
                          .join(au_period, on = ['author_id', 'year'], how = 'inner')
                          .select(*[ col for col in au_inst_period.columns if col not in ['publications','citations'
                                                                                         # ,"avg_rank_source_raw","nr_source_btm_50pct_raw",
                                                                                         # "nr_source_mid_40pct_raw","nr_source_top_20pct_raw","nr_source_top_10pct_raw","nr_source_top_5pct_raw"
                                                                                         # ,'nr_type_article','nr_type_book',
                                                                                         #'nr_type_book-series','nr_language_en','nr_language_fr','citations_type_article'
                                                                                         #,'citations_type_book','citations_type_book-series','citations_language_en','citations_language_fr'
                                                                                         ]],
                                  *[func.mean(func.col(col)/func.col('n_inst')).over(Window.partitionBy('author_id','period_total')).alias(col.replace('_raw', ''))
                                   for col in au_inst_period.columns if col in ['publications_raw','citations_raw'
                                                                                          ,"avg_rank_source_raw","nr_source_btm_50pct_raw",
                                                                                          "nr_source_mid_40pct_raw","nr_source_top_20pct_raw","nr_source_top_10pct_raw","nr_source_top_5pct_raw"
                                                                                #,'nr_type_article','nr_type_book',
                                                                                #         'nr_type_book-series','nr_language_en','nr_language_fr','citations_type_article'
                                                                                #         ,'citations_type_book','citations_type_book-series','citations_language_en','citations_language_fr'
                                                                               ]]
                                 )
                          )
au_inst_period_smoothed.columns

['author_id',
 'inst_id',
 'year',
 'new_inst_period',
 'period_inst',
 'last_y_inst_period',
 'first_y_inst_period',
 'n_inst_y',
 'publications_raw',
 'citations_raw',
 'avg_rank_source_raw',
 'nr_source_btm_50pct_raw',
 'nr_source_mid_40pct_raw',
 'nr_source_top_20pct_raw',
 'nr_source_top_10pct_raw',
 'nr_source_top_5pct_raw',
 'inst_name',
 'inst_type',
 'country',
 'parent',
 'fused',
 'uni_pub',
 'cnrs',
 'entry_year',
 'last_year',
 'all_y_in_FR',
 'years_abroad',
 'main_field',
 'author_name',
 'entry_cohort',
 'publications',
 'citations',
 'avg_rank_source',
 'nr_source_btm_50pct',
 'nr_source_mid_40pct',
 'nr_source_top_20pct',
 'nr_source_top_10pct',
 'nr_source_top_5pct']

In [29]:
au_inst_period_smoothed.write.mode('overwrite').parquet('file:\\' + save_path + 'panel_smoothed.parquet')

In [56]:
save_path

'C:\\Users\\rapha\\Desktop\\\\'

In [8]:
au_inst_period_smoothed = spark.read.parquet('file:\\' + save_path + 'panel_smoothed.parquet')

In [30]:
panel_smoothed_w_past_inst = (au_inst_period_smoothed
                                  .withColumn('all_past_af', func.array_distinct((func.collect_set(func.col('inst_id')).over(Window.partitionBy('author_id').orderBy('year')))))
                                  .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
                                  .withColumn('parent', func.regexp_replace(func.col('parent'), '\[',''))
                                  .withColumn('parent', func.regexp_replace(func.col('parent'), '\]',''))
                                  .withColumn('parent', func.when(func.col('parent').contains(';'), func.split(func.col('parent'), ';'))
                                              .otherwise(func.array(func.col('parent'))))
                                  .withColumn('parent', func.transform(func.col('parent'), lambda x: func.from_json(x, func.schema_of_json('{"parent_id":"","parent_name":"","parent_type":""}'))))
                                  .withColumn('parent', func.array_compact(func.transform(func.col('parent'), lambda x : func.when(x.parent_id != func.col('inst_id'), x))))
                                  .withColumn('all_parent_ids', func.array_compact(func.transform(func.array_distinct(func.flatten(func.collect_set(func.col('parent.parent_id')).over(Window.partitionBy('year')))), lambda x: func.regexp_replace(x, 'https://openalex.org/', ''))))
   
                                  .withColumn('all_past_parents', func.array_distinct(func.flatten(func.collect_set(func.col('all_parent_ids')).over(Window.partitionBy('author_id').orderBy('year')))))
   
                            #      .withColumn('past_af_parent', ( (func.array_contains(func.col('all_past_parents'), func.col('inst_id')))
                            #                                     | (func.arrays_overlap(func.col('all_past_parents'), func.col('all_parent_ids'))) ).cast('int'))
                            #      .withColumn('past_af', ( (func.array_contains(func.col('all_past_af'), func.col('inst_id')))
                            #                                     | (func.arrays_overlap(func.col('all_past_af'), func.col('all_parent_ids'))) ).cast('int'))
                            #  .withColumn('lag_inst_set', func.lag(func.col('inst_set_this_year')).over(Window.partitionBy('author_id').orderBy('year')))
                            #  .withColumn('entrant', func.array_contains(func.col('lag_inst_set'), func.col('inst_id')))
                            #  .select(*au_inst_period_smoothed.columns, "past_af_parent","past_af",'entrant')
                                 )

<>:3: SyntaxWarning: invalid escape sequence '\}'
<>:3: SyntaxWarning: invalid escape sequence '\}'
<>:4: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\}'
<>:3: SyntaxWarning: invalid escape sequence '\}'
<>:4: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\]'
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\1570136931.py:3: SyntaxWarning: invalid escape sequence '\}'
  .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\1570136931.py:3: SyntaxWarning: invalid escape sequence '\}'
  .withColumn('parent', func.regexp_replace(func.col('parent'), '\}\,\{','\}\;\{'))
C:\Users\raphael.wargon\AppData\Local\Temp\ipykernel_9872\1570136931.py:4: SyntaxWarning: invalid escape sequence '\['
  .withColumn('parent', func.regexp_replace(func.col('parent'), '\[',''))
C

In [ ]:
panel_smoothed_w_past_inst.show()

In [32]:
panel_smoothed_w_past_inst.write.mode('overwrite').parquet('file:\\' + save_path + 'panel_smoothed_w_past_inst.parquet')

Py4JJavaError: An error occurred while calling o2097.parquet.
: org.apache.spark.SparkException: Job 70 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2731)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3013)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2994)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2983)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
